In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [4]:
data_usd_euro = pd.read_csv("EUR_USD.csv")
data_usd_euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   1295 non-null   object
 1   Şimdi   1295 non-null   object
 2   Açılış  1295 non-null   object
 3   Yüksek  1295 non-null   object
 4   Düşük   1295 non-null   object
 5   Hac.    1031 non-null   object
 6   Fark %  1295 non-null   object
dtypes: object(7)
memory usage: 70.9+ KB


In [5]:
df_euro = data_usd_euro.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [6]:
df_euro.head(10)

,Tarih,Şimdi
0,16.12.2022,"1,0582"
1,15.12.2022,"1,0626"
2,14.12.2022,"1,0682"
3,13.12.2022,"1,0630"
4,12.12.2022,"1,0535"
5,09.12.2022,"1,0530"
6,08.12.2022,"1,0556"
7,07.12.2022,"1,0505"
8,06.12.2022,"1,0469"
9,05.12.2022,"1,0491"


In [7]:
# We need to change from "," to "." because it will be a float. 
df_euro['Şimdi'] = df_euro['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df_euro

,Tarih,Şimdi
0,16.12.2022,1.0582
1,15.12.2022,1.0626
2,14.12.2022,1.0682
3,13.12.2022,1.0630
4,12.12.2022,1.0535
...,...,...
1290,05.01.2018,1.2030
1291,04.01.2018,1.2068
1292,03.01.2018,1.2014
1293,02.01.2018,1.2059


In [8]:
df_euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1295 entries, 0 to 1294
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tarih   1295 non-null   object 
 1   Şimdi   1295 non-null   float64
dtypes: float64(1), object(1)
memory usage: 20.4+ KB


In [9]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = df_euro['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.0582 1.0626 1.0682 1.063  1.0535] 1.053
[1.0626 1.0682 1.063  1.0535 1.053 ] 1.0556
[1.0682 1.063  1.0535 1.053  1.0556] 1.0505
[1.063  1.0535 1.053  1.0556 1.0505] 1.0469
[1.0535 1.053  1.0556 1.0505 1.0469] 1.0491
[1.053  1.0556 1.0505 1.0469 1.0491] 1.0538
[1.0556 1.0505 1.0469 1.0491 1.0538] 1.0522
[1.0505 1.0469 1.0491 1.0538 1.0522] 1.0405
[1.0469 1.0491 1.0538 1.0522 1.0405] 1.0327
[1.0491 1.0538 1.0522 1.0405 1.0327] 1.0337
[1.0538 1.0522 1.0405 1.0327 1.0337] 1.0395
[1.0522 1.0405 1.0327 1.0337 1.0395] 1.0408
[1.0405 1.0327 1.0337 1.0395 1.0408] 1.0395
[1.0327 1.0337 1.0395 1.0408 1.0395] 1.0302
[1.0337 1.0395 1.0408 1.0395 1.0302] 1.0241
[1.0395 1.0408 1.0395 1.0302 1.0241] 1.0324
[1.0408 1.0395 1.0302 1.0241 1.0324] 1.036
[1.0395 1.0302 1.0241 1.0324 1.036 ] 1.0392
[1.0302 1.0241 1.0324 1.036  1.0392] 1.0348
[1.0241 1.0324 1.036  1.0392 1.0348] 1.0325
[1.0324 1.036  1.0392 1.0348 1.0325] 1.0352
[1.036  1.0392 1.0348 1.0325 1.0352] 1.0208
[1.0392 1.0348 1.0325 1.0352 1.020

In [10]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df_euro['Şimdi'][0],df_euro['Şimdi'][1],df_euro['Şimdi'][2],df_euro['Şimdi'][3],df_euro['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
41/41 [==============================] - 2s 18ms/step - loss: 0.2693
Epoch 2/200
41/41 [==============================] - 1s 16ms/step - loss: 0.0018
Epoch 3/200
41/41 [==============================] - 0s 8ms/step - loss: 9.1322e-05
Epoch 4/200
41/41 [==============================] - 0s 8ms/step - loss: 7.5720e-05
Epoch 5/200
41/41 [==============================] - 0s 8ms/step - loss: 7.5420e-05
Epoch 6/200
41/41 [==============================] - 0s 8ms/step - loss: 7.4081e-05
Epoch 7/200
41/41 [==============================] - 0s 10ms/step - loss: 7.3958e-05
Epoch 8/200
41/41 [==============================] - 1s 15ms/step - loss: 7.4558e-05
Epoch 9/200
41/41 [==============================] - 0s 9ms/step - loss: 7.4984e-05
Epoch 10/200
41/41 [==============================] - 0s 8ms/step - loss: 7.3955e-05
Epoch 11/200
41/41 [==============================] - 0s 8ms/step - loss: 7.4731e-05
Epoch 12/200
41/41 [==============================] - 0s 8ms/step - loss: 7.33

In [11]:
print("Prediction:",round(yhat[0][0],3), "Euro/USD")# onluk olarak yukarı yuvarlandı.

Prediction: 1.067 Euro/USD
